# Merging datasets

Author: Andrea Mock

After having obtained data from the Hive and Linkedin, we have to merge the datasets. Since each dataset contains different information, we have to be cognizant of the differences when merging, eventually dropping, renaming and creating new columns.  

In [2]:
# loading necessary libraries
import pandas as pd
import json

## Part 1: Read in the data
In the first step we read in the datasets. The linkedin data is saved in the CSV file titled 'majors_v2.csv' and the hive data is saved as a JSON file titled 'alums_hive.json'. 

In [3]:
# load in linkedin data
df1 = pd.read_csv('majors_v2.csv', index_col = 0)

In [99]:
#df1.head()

In [5]:
# Opening JSON file with Hive data
with open('alums_hive.json', 'r') as openfile: 
    # Reading from json file 
    json_object = json.load(openfile) 

In [6]:
df2 = pd.DataFrame(json_object)

## Part 2: Cleaning the Hive data
Unlike the Linkedin data, which was mostly already cleaned beforehand, the Hive data still requires cleaning. We thus extract information including graduation year, location, and current occupation.

In [7]:
# extract an alumna's name and save it in a new column
df2['name'] = df2['person-card__name-block'].apply( lambda x: x[0].split('’')[0].strip())

In [8]:
def extractGradYear(person): 
    """
    extracts the grad year of an alum, an adds either 20 or 19 at the beginning to create full graduation years 
    ie. 2004 or 1998, if no grad year available return -1
    """
    info = person[0].split('’')
    if len(info) > 1:
        if (int(info[1]) >= 0 and int(info[1]) <= 20):
            return '20' + info[1]
        else:
            return '19' +  info[1]
    else:
        return -1

In [9]:
# extract grad year and save it in a new column
df2['grad_year'] = df2['person-card__name-block'].apply( lambda x: extractGradYear(x))

In [10]:
# extract location and save it into a new column
df2['location'] = df2['person-card__details'].apply( lambda x: x[1])

In [11]:
# extract occupation and save it into a new column
df2['current_occupation'] = df2['person-card__details'].apply( lambda x: x[0])

Our data asides from occupation, location and graduation year, includes more information including education. These are just a few examples of types of information our dataset contains.

In [12]:
# showing what all occupations string includes 
df2['person-details-container__info-line'].apply(lambda x: x[1])[815]

'Unemployed and Actively Looking, University of Delaware, Gamehelper, DATASTAR, INC, STS MARKET RESEARCH, Massachusetts Institute of Technology, Boston College, Wellesley College Art Department, University of Florida'

In [13]:
# only occupation example
df2.iloc[815]['current_occupation']

'Writer, Editor, Database/Query Writer, Web Developer'

In [14]:
# all of education listed
df2['person-details-container__info-line'].apply(lambda x: x[0])[0]

"Wellesley College, Bachelor's Degree, 2012, Art History, Courtauld Institute of Art, Master's Degree, 2013, History of Art & Architecture, Bard Graduate Center, Master's Degree, 2015, Decorative Arts & Design History, Harvard University, PhD - Doctor of Philosophy, History of Art & Architecture"

## Part 3: Checking for duplicates
After having extracted information such as name and graduation year, we can make a comparison with the data extracted from linkedin and see if there is any overlap. to do so we create subdata frames that only include name and graduation year, two identifiers that can help us identify overlaps between teh datasets.

In [15]:
# data from dataset from linkedin 
df1_names = df1[['name', 'grad_year']]

In [98]:
#df1_names

In [17]:
# names and graduation year from the hive
df2_names = df2[['name', 'grad_year']]

In [97]:
df2_names

In [19]:
# merge both datasets 
all_names = pd.concat([df1_names, df2_names], ignore_index=True)

In [20]:
# see how many duplicates exist
all_names['name'].duplicated().value_counts()

False    1334
True       95
Name: name, dtype: int64

In [21]:
# create a true false indicator vector that has a value true if the value appears more than once, false otherwise
duplicates = all_names['name'].duplicated()

In [96]:
all_names[duplicates] # in total their are 95 duplicated values

## Part 4: Continue cleaning data from the Hive
In a first step, after having extracted some information and added columns to the dataset containing the Hive data is to drop the unnecessary colums to avoid confusion and allow for easier cleaning of the rest of the data.

In [23]:
# drop column
df2 = df2.drop(['person-card__name-block'], axis=1)

In [24]:
df2.columns

Index(['person-card__details', 'person-details-container__info-line', 'name',
       'grad_year', 'location', 'current_occupation'],
      dtype='object')

In [25]:
# reindesx dataframe
df2 = df2.reindex(columns=['name', 'grad_year', 'location', 'current_occupation', 'person-card__details', 'person-details-container__info-line'])

In [655]:
# save first version of data
df2.to_pickle('hive_alums')

In [26]:
# extract the employer from hive data
df2['current_employer'] = df2['person-details-container__info-line'].apply(lambda x : x[1].split(',')[0])

In [27]:
# extract education info from hive data
df2['education'] = df2['person-details-container__info-line'].apply(lambda x : x[0])

In [28]:
df2.iloc[812] # sampe of one current dataentry 

name                                                                        Breslin Bell
grad_year                                                                           2018
location                                                                   Wellesley, MA
current_occupation                                Art Historian, Educator and Printmaker
person-card__details                   [Art Historian, Educator and Printmaker, Welle...
person-details-container__info-line    [Wellesley College, Bachelor's Degree, 2018, A...
current_employer                                       Davis Museum at Wellesley College
education                              Wellesley College, Bachelor's Degree, 2018, Ar...
Name: 812, dtype: object

In [29]:
df2.iloc[812]['person-details-container__info-line'][1]

'Davis Museum at Wellesley College'

In [30]:
# split the education data and save it into a new column
edu1 = df2['education'].apply(lambda x: x.split(','))

In [31]:
df2['clean_edu1'] = edu1

Although we filtered our dataset for Wellesley alums, there are still a few professors and others that ended up in the dataset despite this. Thus we check if all people in our dataset every attended wellesley and remove those that never attended Wellesley. 

In [32]:
# indicator of whether someone attended Wellesley
isWellesleyGrad = df2['education'].apply(lambda x: 'Wellesley' in x)

In [33]:
# only include Wellesley alums not people who work at wellesley
wellesley_alums = df2[isWellesleyGrad].reset_index(drop=True)

Now after having filtered to only include those who have Wellesley listed as the first thing in their education section and those who have something else listed first. 

In [34]:
# break down data into alums that have Wellesley listed as their first education entry 
edu2 = wellesley_alums['clean_edu1'].apply(lambda x: x[0] == 'Wellesley College')

In [35]:
# break down data into alums that have Wellesley listed as their first education entry and those who don't
w1 = wellesley_alums[edu2].reset_index(drop=True)
w2 = wellesley_alums[~edu2].reset_index(drop=True)

In [40]:
w1['majors_cleaning'] = w1['clean_edu1'].apply(lambda x: x[1:])

Since the education data entry in most cases includes wellesley, bacehlors etc. before the major we first want to remvoe these to grab only the majors. However, since some included their degree and some did not we cannot jsut remove a whole list entry instead we only remove those entries that have a degree listed.

In [37]:
def el0(item):
    """
    return the first element of a list otherwise if the entry is empty, return empty string instead
    """
    if len(item) > 0:
        return item[0]
    return ''

In [38]:
def deleteBachelor(education):
    """
    
    """
    bachelorVar = ["Bachelor's Degree", "BA", "", "Bachelor of Arts - BA", "Bachelor’s Degree", 
                   "Bachelor of Arts (B.A.)", "Bachelor of Arts (BA)",'B.A.', 'Graduate Certificate',
                  'Bachelors', "Bachelor's degree"]
    if len(education) > 0:
        if education[0].strip() in bachelorVar:
            return education[1:]
        else:
            return education
    return []

In [41]:
w1['m1'] = w1['majors_cleaning'].apply(lambda x: deleteBachelor(x))

In [43]:
def cleanGradYear(education):
    """
    retrieve the grad year and save it as an int, if grad year is not included return -1
    """
    if len(education) >0:
        try:
            return int(education[0])
        except:
            return -1
    return -1

In [44]:
w1['grad_year_c1'] = w1['m1'].apply(lambda x: cleanGradYear(x))

In [45]:
def deleteGradYear(education):
    """
    get rid of the graduation year in entry if the graduation year is included in an entry
    """
    if len(education) >0:
        try:
            gradYear = int(education[0])
            if len(education) > 1:
                return education[1:]
            return []
        except:
            return education
    return []

In [53]:
w1['m2'] = w1['m1'].apply(lambda x: deleteGradYear(x))

In [54]:
import re

In [55]:
def cleaning2(major): 
    """
    normalize the majors, so that there is uniform expression of majors 
    
    """
    if ('Computer' in major or 'computer' in major):
        return 'Computer Science'
    elif ('Cognitive' in major or 'Linguistic' in major):
        return 'Cognitive Science'
    elif ('Internat' in major):
        return 'International Relations'
    elif 'Biolo' in major:
        return 'Biology'
    elif 'Classic' in major:
        return 'Classics'
    elif ('English' in major or 'Writing' in major):
        return 'English'
    elif ('Geology' in major or 'Geoscience' in major):
        return 'Geology'
    elif ('Studio Art' in major or 'Art Studies' in major or 'Fine Art' in major or 'Art' == major):
        return 'Studio Art'
    elif ('Math' in major):
        return 'Math'
    elif ('Education' in major):
        return 'Education'
    elif ('Russian' in major):
        return 'Russian'
    elif ('Logic' in major or 'Philosophy' in major):
        return 'Philosophy'
    elif ('Cinema' in major):
        return 'CAMS'
    elif ('music' in major.lower()):
        return 'Music'
    elif ('Econom' in major):
        return 'Economics'
    elif 'Biochemistry' in major:
        return 'Biochemistry'
    elif 'Environment' in major:
        return 'Environmental Studies'
    elif 'Chinese' in major:
        return 'Chinese'
    elif 'Japanese' in major:
        return 'Japanese'
    elif 'German' in major:
        return 'German'
    elif ('Comparative Lit' in major or 'Literature' in major):
        return 'Comparative Literature'
    elif 'American Studies' in major:
        return 'American Studies'
    elif 'Astrophysics' in major:
        return 'Astrophysics'
    elif ('Justice Studies' in major or 'Peace' in major):
        return 'PAJS'
    elif ('Art' in major and 'History' in major):
        return 'Art History'
    elif 'Chemical Physics' in major:
        return 'Chemical Physics'
    elif 'Theatre Studies' in major:
        return 'Theatre Studies'
    elif 'Architec' in major:
        return 'Architecture'
    elif 'Italian' in major:
        return 'Italian Studies'
    elif ('Anthro' in major or 'Archeo' in major):
        return 'Anthropology'
    return major

In [56]:
def cleanMajor1(education):
    """
    majors such as mas and wgst get changed to their shortened name if given as an input other entries stay unchanged
    """
    if (len(education) > 0):
        words = education[0]
        if (type(words) == str):
            words = words.strip()
            words = re.sub("Women's and Gender Studies", "WGST", words)
            words = re.sub("Women & Gender Studies", "WGST", words)
            words = re.sub("Women’s and Gender Studies", "WGST", words)
            words = re.sub("Women's Studies", "WGST", words)
            words = re.sub("Women's & Gender Studies", "WGST", words)
            words = re.sub("Media Arts and Sciences", "MAS", words)
            words = re.sub('Media Arts & Sciences', "MAS", words)
            words = re.sub("Media Arts and Science", "MAS", words)
            words = re.sub("Media Arts & Sicences", "MAS", words)
            words = re.sub('Indiv-Media Arts & Sci', "MAS", words)
            words = re.sub('Media Arts and Computer Science','MAS', words)
            words = re.sub('Computer Science and Cinema and Media Studies','MAS', words)
            words = re.sub('Media Art & Computer Science','MAS', words)
            words = re.sub('Media Arts and Scienc','MAS', words)
            words = re.sub('Media Arts & Science','MAS', words)
            words = re.sub('Computer Science  and Media Arts','MAS', words)
            words = re.sub('Computer Science and Digital Arts','MAS', words)
            words = cleaning2(words)
            education[0] = words
    return education

In [57]:
w1['major1'] =w1['m2'].apply(lambda x:el0(cleanMajor1(x)))

In [58]:
def cleaning3(education):
    major_list = ['Africana Studies',
 'American Studies',
 'Anthropology',
 'Arabic',
 'Architecture',
 'Art History',
 'Asian-American Studies',
 'Astronomy',
 'Biochemistry',
 'Biology',
 'CAMS',
 'Chemistry',
 'Chinese',
 'Classics',
 'Cognitive Science',
 'Comparative Literature',
 'Computer Science',
 'Data Science',
 'East Asian Studies',
 'Economics',
 'Education',
 'English',
 'Environmental Studies',
 'French',
 'Geology',
 'German',
 'History',
 'History and Religion',
 'International Relations',
 'Italian Studies',
 'Japanese',
 'Jewish Studies',
 'Latin',
 'MAS',
 'Math',
 'Medieval and Renaissance Studies',
 'Middle Eastern Studies',
 'Music',
 'Neuroscience',
 'Philosophy',
 'Physics',
 'Political Science',
 'Psychology',
 'Russian',
 'Sociology',
 'Spanish',
 'Studio Art',
 'WGST',
       'English', 'Chemistry', 'Math', 'Sociology', 'Russian', 'Geology',
       'Political Science', '', 'Language Studies', 'French', 'Economics',
       'American Studies', 'Art Studio', 'Neuroscience',
       'Computer Science', 'Biochemistry', 'Biology', 'CAMS',
       'Philosophy', 'Cognitive Science', 'Spanish',
       'Indiv-Urban Studies', 'Japanese', 'Astronomy', 'History',
       'Chinese', 'Psychology',
       'East Asian Studies', 'Music', 'Theatre Studies',
       'International Relations', 'Comparative Literature',
       'Anthropology', 'Chemical Physics', 'Studio Art',
       'Environmental Studies', 'Architecture', 'WGST', 'German',
       'Italian', 'PAJS',
       'French Cultural Studies', 'Psychology and Spanish',
       'Africana Studies', 'Astrophysics',
       'Bioinformatics', 'Italian Studies', 'French Studies', 'Latin',
       'Classics']
    if len(education) > 0:
        major =education[0] 
        if type(major) == str:
            if major in major_list:
                return major
    return ''

In [59]:
def cleaning4(education):
    major_list = ['Africana Studies',
 'American Studies',
 'Anthropology',
 'Arabic',
 'Architecture',
 'Art History',
 'Asian-American Studies',
 'Astronomy',
 'Biochemistry',
 'Biology',
 'CAMS',
 'Chemistry',
 'Chinese',
 'Classics',
 'Cognitive Science',
 'Comparative Literature',
 'Computer Science',
 'Data Science',
 'East Asian Studies',
 'Economics',
 'Education',
 'English',
 'Environmental Studies',
 'French',
 'Geology',
 'German',
 'History',
 'History and Religion',
 'International Relations',
 'Italian Studies',
 'Japanese',
 'Jewish Studies',
 'Latin',
 'MAS',
 'Math',
 'Medieval and Renaissance Studies',
 'Middle Eastern Studies',
 'Music',
 'Neuroscience',
 'Philosophy',
 'Physics',
 'Political Science',
 'Psychology',
 'Russian',
 'Sociology',
 'Spanish',
 'Studio Art',
 'WGST',
       'English', 'Chemistry', 'Math', 'Sociology', 'Russian', 'Geology',
       'Political Science', '', 'Language Studies', 'French', 'Economics',
       'American Studies', 'Art Studio', 'Neuroscience',
       'Computer Science', 'Biochemistry', 'Biology', 'CAMS',
       'Philosophy', 'Cognitive Science', 'Spanish',
       'Indiv-Urban Studies', 'Japanese', 'Astronomy', 'History',
       'Chinese', 'Psychology',
       'East Asian Studies', 'Music', 'Theatre Studies',
       'International Relations', 'Comparative Literature',
       'Anthropology', 'Chemical Physics', 'Studio Art',
       'Environmental Studies', 'Architecture', 'WGST', 'German',
       'Italian', 'PAJS',
       'French Cultural Studies', 'Psychology and Spanish',
       'Africana Studies', 'Astrophysics',
       'Bioinformatics', 'Italian Studies', 'French Studies', 'Latin',
       'Classics']
    if len(education) > 0:
        major = education[0] 
        if type(major) == str:
            if major in major_list:
                if len(education) > 1:
                    return education[1:]
            else:
                return []
    return []

In [60]:
w1['major1'] = w1['m2'].apply(lambda x:cleaning3(cleanMajor1(x)))

In [61]:
w1['m3'] = w1['m2'].apply(lambda x:cleaning4(cleanMajor1(x)))

In [62]:
w1['major2'] = w1['m3'].apply(lambda x:cleaning3(cleanMajor1(x)))

In [63]:
# look at columns
w1.columns

Index(['name', 'grad_year', 'location', 'current_occupation',
       'person-card__details', 'person-details-container__info-line',
       'current_employer', 'education', 'clean_edu1', 'majors_cleaning', 'm1',
       'grad_year_c1', 'm2', 'major1', 'm3', 'major2'],
      dtype='object')

After having added columns for cleaning we can drop these as they are no longer needed.

In [64]:
w1['grad_school'] = w1['m3'].apply(lambda x: len(x) >1)

In [95]:
#w1

In [66]:
w1 = w1.drop(['m1', 'm2', 'clean_edu1', 'majors_cleaning','education', ], axis=1)

In [67]:
def graduationDate(person):
    if person['grad_year'] != -1:
        return person['grad_year']
    elif person['grad_year_c1'] != -1:
        return person['grad_year_c1']
    else:
        return -1

In [68]:
w1['graduation'] = w1.apply(lambda x: graduationDate(x), axis =1)

In [69]:
# drop columns used to clean graduation date
w1 = w1.drop(['grad_year', 'grad_year_c1'], axis =1)

In [70]:
def findWellesleyEdu(edu):
    counter = 0
    for item in edu:
        if 'Wellesley' in item:
            return edu[counter:]
        counter += 1
    return []

In [71]:
w2['majors_cleaning'] = w2['clean_edu1'].apply(lambda x: findWellesleyEdu(x))

In [72]:
def deleteWellesley(edu):
    if len(edu) >0:
        if 'Wellesley' in edu[0]:
            if len(edu) > 1:
                return edu[1:]
            return []
    return []

In [73]:
w2['majors_cleaning'].apply(lambda x: el0(deleteWellesley(x))).unique()

array([" Bachelor's Degree", ' Bachelor’s Degree',
       ' Bachelor of Arts (B.A.)', ' Bachelor of Arts (BA)',
       ' Bachelor of Arts - BA', ' 2009', " Bachelor's degree"],
      dtype=object)

In [74]:
w2['m1'] = w2['majors_cleaning'].apply(lambda x: deleteBachelor(deleteWellesley(x)))

In [75]:
w2['grad_year_c1'] = w2['m1'].apply(lambda x: cleanGradYear(x))

In [76]:
w2['m2'] = w2['m1'].apply(lambda x: deleteGradYear(x))

In [77]:
w2['major_1'] = w2['m2'].apply(lambda x:cleaning3(cleanMajor1(x)))

In [78]:
w2['m3'] = w2['m2'].apply(lambda x:cleaning4(cleanMajor1(x)))

In [79]:
w2['major_2'] = w2['m3'].apply(lambda x:cleaning3(cleanMajor1(x)))

In [80]:
w2['m4'] = w2['m3'].apply(lambda x:cleaning4(cleanMajor1(x)))

In [81]:
w2['graduation'] = w2.apply(lambda x: graduationDate(x), axis =1)

In [82]:
# drop columns used to clean graduation date
w2 = w2.drop(['grad_year', 'grad_year_c1'], axis =1)

In [83]:
w2 = w2.drop(['m1', 'm2', 'm3', 'clean_edu1', 'majors_cleaning','education', ], axis=1)

In [84]:
w2['grad_school'] = w2['m4'].apply(lambda x: len(x) >1)

In [85]:
w1 = w1.rename(columns={'current_occupation': 'headline', 'm3': 'majorCleaning'})

In [86]:
w2 = w2.rename(columns={'current_occupation': 'headline', 'm4': 'majorCleaning'})

In [87]:
w1_clean = w1[['name', 'location', 'headline', 'person-details-container__info-line', 'current_employer', 'majorCleaning',
   'major1', 'major2', 'graduation', 'grad_school']]

In [88]:
w2 = w2.rename(columns={'major_2': 'major2', 'major_1': 'major1'})

In [89]:
w2_clean = w2[['name', 'location', 'headline', 'person-details-container__info-line', 'current_employer', 'majorCleaning',
   'major1', 'major2', 'graduation', 'grad_school']]

In [90]:
hive_all = pd.concat([w1_clean, w2_clean]).reset_index(drop=True)

In [94]:
hive_all # newly merged and cleaned data

In [713]:
hive_all.to_pickle('hive_data_cleaned') # save newly cleaned hive data

In [714]:
df1.columns

Index(['name', 'headline', 'summary', 'location', 'skills', 'education',
       'jobs', 'degree', 'study_range', 'grad_year', 'title', 'company',
       'description', 'url', 'end', 'start', 'attended_grad_school', 'major1',
       'major2', 'major3'],
      dtype='object')

In [715]:
hive_all.columns

Index(['name', 'location', 'headline', 'person-details-container__info-line',
       'current_employer', 'majorCleaning', 'major1', 'major2', 'graduation',
       'grad_school'],
      dtype='object')

Now that we have most of the data from the hive as well as our linkedin data cleaned we want to merge both datasets and create one that encompasses the info from both. Our first step is thus to look at the different columns contained in each, getting rid of unnecessary columns and create one giant dataset with matching column names.

Our first data set has the following column names: 
'name', 'headline', 'summary', 'location', 'skills', 'education','jobs', 'degree', 'study_range', 'grad_year', 'title', 'company','description', 'url', 'end', 'start', 'attended_grad_school', 'major1', 'major2', 'major3'

Our hive dataset includes the following column names:
'name', 'location', 'headline', 'person-details-container__info-line', 'current_employer', 'majorCleaning', 'major1', 'major2', 'graduation', 'grad_school'

We thus see that there is some overlap as well as certain columns that can be deleted.

The columns that match up are name, location, headline, company/current_employer, major1, major2, (major3), graduation/end (graduation year)

Other columns that we also will inculde that are only in our first dataset from linkedin are url, description, skills and summary. 

We thus resort to creating one dataframe that includes all the information that both contain, and create an additional dataframe only focusing on particular columns that are only included in our linkedin dataset. 

In [716]:
smaller_data_hive = hive_all[['name', 'location', 'headline', 'current_employer', 'graduation', 'major1', 'major2']]
smaller_data_hive = smaller_data_hive.rename(columns={'graduation': 'grad_year'})

In [92]:
smaller_data_hive.head()

In [718]:
smaller_data_linkedin = df1[['name', 'location', 'headline', 'company', 'title', 'grad_year', 'major1', 'major2', 'major3']]
smaller_data_linkedin = smaller_data_linkedin.rename(columns={'company': 'current_employer'})

In [93]:
smaller_data_linkedin.head()

In [720]:
all_data_merged = pd.concat([smaller_data_linkedin, smaller_data_hive], ignore_index=True)

In [721]:
all_data_merged.to_csv('all_alum_data.csv') 

In [722]:
anon_df = all_data_merged[[ 'location', 'headline', 'current_employer', 'title',
       'grad_year', 'major1', 'major2', 'major3']]

In [723]:
anon_df.to_csv('alum_data_anon.csv') # save data without personal names for sharing 